In [4]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from sklearn.datasets import make_blobs
import random
import json
import time
from tabulate import tabulate

from kdtree import *
from helper import *

In [5]:
savefile = "mst_data2.json"
to_plot = False
no_points = 10000
no_centres = 100

In [23]:
X, Y = make_blobs(n_samples=no_points, centers=no_centres, random_state=42)
points = [(x, y) for x, y in X]

In [24]:
cordmap = {point: i for i, point in enumerate(points)}

In [25]:
def dcran(cordmap):
    tree = KDTree()
    tree.root = tree.build(points)
    G = nx.Graph()
    for coord, index in cordmap.items():
        G.add_node(index, pos=coord)

    k = 2
    connected_components = 1
    while True:
        maxdis = min(len(cordmap) , 2*factorial(k))
        for pointi, i in cordmap.items():
            pointj = ith_nearest_neighbor(tree, pointi, k)
            dis = euclidean_distance(pointi, pointj)
            G.add_edge(i, cordmap[pointj], weight=dis)
            # print(k ,  i , cordmap[pointj] , dis )
            # randis = random.randint(k + 1, maxdis)
            randis = maxdis
            # randis = (cusrandis(k + 1 , maxdis))
            # print(randis , k)
            pointj = ith_nearest_neighbor(tree, pointi, randis)
            dis = euclidean_distance(pointi, pointj)
            G.add_edge(i, cordmap[pointj], weight=dis)
            # print(k ,  i , cordmap[pointj] , dis , randis)
        ccount = count_clusters(G)
        print(k, ccount)
        k += 1
        if ccount == 1:
            break

    return G

In [26]:
dcran_start_time = time.time()
kc = dcran(cordmap)

2 81
3 4
4 3
5 3
6 2
7 2
8 1


In [27]:
graphify(kc, to_plot)

In [28]:
G = kc
num_nodes = G.number_of_nodes()
conedge = num_edges = G.number_of_edges()

# Calculate the sum of all edge weights
total_weight = sum(data["weight"] for u, v, data in G.edges(data=True))

print(f"Total number of nodes: {num_nodes}")
print(f"Total number of edges: {num_edges}")
print(f"Total sum of edge weights: {total_weight}")

Total number of nodes: 10000
Total number of edges: 90645
Total sum of edge weights: 37051.450303763624


In [29]:
mst = nx.minimum_spanning_tree(G, algorithm="kruskal", weight="weight")
graphify(mst, to_plot)

print("Edges in the MST:", list(mst.edges(data=True)), sep="\n")

Edges in the MST:
[(0, 8138, {'weight': 0.010169869429594115}), (0, 3550, {'weight': 0.07392726409332272}), (1, 2089, {'weight': 0.04348236333736803}), (1, 9741, {'weight': 0.05522285775413705}), (2, 6453, {'weight': 0.07262466559938485}), (2, 1604, {'weight': 0.2620234158454776}), (3, 5095, {'weight': 0.16984256307993112}), (3, 7632, {'weight': 0.19648943554581091}), (4, 1246, {'weight': 0.04663897950520702}), (4, 119, {'weight': 0.06925652203703091}), (5, 1452, {'weight': 0.030418270890687774}), (5, 6027, {'weight': 0.0316092672447535}), (6, 8851, {'weight': 0.024472499746885392}), (6, 3949, {'weight': 0.044960538151866444}), (7, 1344, {'weight': 0.0804274334000805}), (7, 4404, {'weight': 0.08072211241882604}), (8, 3039, {'weight': 0.039511326230144056}), (8, 3849, {'weight': 0.07439498242190946}), (9, 4156, {'weight': 0.0494106087328281}), (10, 1749, {'weight': 0.09205842540615253}), (11, 1639, {'weight': 0.03695558255195429}), (11, 1788, {'weight': 0.16407017699610107}), (12, 5803,

In [30]:
G = mst
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()


foundw = total_weight = sum(data["weight"] for u, v, data in G.edges(data=True))

print(f"Total number of nodes: {num_nodes}")
print(f"Total number of edges: {num_edges}")
print(f"Total sum of edge weights: {total_weight}")
dcran_end_time = time.time()
dcran_elapsed_time = dcran_end_time - dcran_start_time

Total number of nodes: 10000
Total number of edges: 9999
Total sum of edge weights: 866.0209239473647


In [31]:
stmst_start_time = time.time()
G = nx.Graph()

for pointi, i in cordmap.items():
    G.add_node(i, pos=pointi)
    for pointj, j in cordmap.items():
        if i != j:
            dis = euclidean_distance(pointi, pointj)
            G.add_edge(i, j, weight=dis)

In [32]:
mst = nx.minimum_spanning_tree(G, algorithm="kruskal", weight="weight")
graphify(mst, to_plot)

In [33]:
G = mst
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

# Calculate the sum of all edge weights
realw = total_weight = sum(data["weight"] for u, v, data in G.edges(data=True))

print(f"Total number of nodes: {num_nodes}")
print(f"Total number of edges: {num_edges}")
print(f"Total sum of edge weights: {total_weight}")
stmst_end_time = time.time()
stmst_elapsed_time = stmst_end_time - stmst_start_time

Total number of nodes: 10000
Total number of edges: 9999
Total sum of edge weights: 865.3329904388614


In [34]:
percentage_error = ((foundw - realw) / realw) * 100

# Format the output for readability
formatted_output = f"Real Weight: {realw}  Found Weight: {foundw}  no of Edge: {conedge} Percentage Error: {percentage_error:.2f}%"
formatted_output

'Real Weight: 865.3329904388614  Found Weight: 866.0209239473647  no of Edge: 90645 Percentage Error: 0.08%'

In [35]:
with open(savefile, "r") as f:
    loaded_data = json.load(f)
print(loaded_data)
currres = []
speedup = round((stmst_elapsed_time  / dcran_elapsed_time) , 2)
loaded_data.append(
    [
        no_points,
        no_centres,
        foundw,
        realw,
        conedge,
        round(100 - percentage_error , 3),
        round(dcran_elapsed_time , 2),
        round(stmst_elapsed_time ,2),
        speedup
    ]
)

# Save the updated dictionary back to the JSON file
with open(savefile, "w") as f:
    json.dump(loaded_data, f)

[[1000, 10, 262.4419314705513, 258.819134723451, 8252, 98.6, 1.83, 4.38, 2.4]]


In [6]:
with open(savefile, "r") as f:
    loaded_data = json.load(f)
print(loaded_data)

[[1000, 10, 262.4419314705513, 258.819134723451, 8252, 98.6, 1.83, 4.38, 2.4], [10000, 10, 866.0209239473647, 865.3329904388614, 90645, 99.921, 28.58, 551.35, 19.29], [10000, 100, 1342.6324139702915, 1337.4478711909765, 27591, 99.612, 3.24, 510.53, 157.39], [10000, 100, 1342.6324139702915, 1337.4478711909762, 27591, 99.612, 2.48, 632.89, 255.22], [10000, 1000, 1459.7289235192638, 1455.1233116849917, 27457, 99.683, 2.5, 565.44, 226.17], [10000, 10, 866.0209239473647, 865.3329904388614, 90645, 99.92050071867081, 24.179672241210938, 617.7955191135406, 25.55], [10000, 50, 1260.1226131964793, 1255.8337705131123, 27661, 99.65848643474409, 2.1823902130126953, 513.3643357753754, 235.23], [10000, 10, 866.0209239473647, 865.3329904388614, 90645, 99.92050071867081, 24.11609435081482, 543.8352909088135, 22.55], [10000, 10, 875.7172119877604, 865.3329904388614, 38587, 98.79997392175785, 16.157432556152344, 646.0387144088745, 39.98], [10000, 100, 1342.6324139702872, 1337.4478711909717, 27591, 99.612

In [7]:
headers = [
    "Points",
    "Centres",
    "DCRAN Wt",
    "kruskal Wt",
    "Edge count",
    "Acc(%)",
    "DCRAN Time (s)",
    "STMST Time (s)",
    "Speedup"
]

# Format the data as a table using tabulate
table_str = tabulate(loaded_data, headers, tablefmt="pipe", floatfmt=(".0f", ".0f", ".1f", ".1f", ".0f", ".1f", ".2f", ".2f", ".2f"))
print(table_str)




|   Points |   Centres |   DCRAN Wt |   kruskal Wt |   Edge count |   Acc(%) |   DCRAN Time (s) |   STMST Time (s) |   Speedup |
|---------:|----------:|-----------:|-------------:|-------------:|---------:|-----------------:|-----------------:|----------:|
|     1000 |        10 |      262.4 |        258.8 |         8252 |     98.6 |             1.83 |             4.38 |      2.40 |
|    10000 |        10 |      866.0 |        865.3 |        90645 |     99.9 |            28.58 |           551.35 |     19.29 |
|    10000 |       100 |     1342.6 |       1337.4 |        27591 |     99.6 |             3.24 |           510.53 |    157.39 |
|    10000 |       100 |     1342.6 |       1337.4 |        27591 |     99.6 |             2.48 |           632.89 |    255.22 |
|    10000 |      1000 |     1459.7 |       1455.1 |        27457 |     99.7 |             2.50 |           565.44 |    226.17 |
|    10000 |        10 |      866.0 |        865.3 |        90645 |     99.9 |            24.18 |

In [38]:
print("Line 1\nLine 2")


Line 1
Line 2
